In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
data_train_avg = pd.read_csv("../Data/train_avg_age.csv")
data_valid_avg = pd.read_csv("../Data/val_avg_age.csv")
data_test_avg = pd.read_csv("../Data/test_avg_age.csv")

data_train_knn = pd.read_csv("../Data/train_knn_age.csv")
data_valid_knn = pd.read_csv("../Data/val_knn_age.csv")
data_test_knn = pd.read_csv("../Data/test_knn_age.csv")

In [3]:
data_train_avg

,affiliate_channel,affiliate_provider,age,country_destination,dac_month,first_affiliate_tracked,first_browser,first_device_type,gender,language,signup_app,signup_flow,signup_method,tfa_month
0,2.0,4.0,37.0,11,7,6.0,43.0,3.0,0.0,6.0,2.0,0.0,0.0,7
1,2.0,4.0,37.0,11,11,0.0,43.0,3.0,0.0,6.0,2.0,0.0,0.0,11
2,7.0,8.0,37.0,4,2,0.0,8.0,3.0,1.0,6.0,2.0,0.0,0.0,2
3,2.0,4.0,27.0,4,4,0.0,8.0,6.0,1.0,6.0,2.0,0.0,1.0,4
4,5.0,1.0,64.0,11,2,3.0,22.0,6.0,2.0,6.0,2.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170755,2.0,4.0,37.0,6,8,6.0,8.0,6.0,0.0,6.0,2.0,0.0,0.0,8
170756,2.0,4.0,26.0,11,12,0.0,43.0,3.0,1.0,6.0,2.0,0.0,1.0,12
170757,2.0,4.0,58.0,4,9,6.0,8.0,6.0,1.0,6.0,2.0,0.0,0.0,9
170758,2.0,4.0,58.0,4,9,0.0,43.0,3.0,1.0,6.0,2.0,0.0,0.0,9


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device.".format(device))

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_chain = nn.Sequential(
            nn.Linear(13, 512),
            nn.ReLU(),
            nn.LayerNorm(512),
            #nn.Linear(512, 512),
            #nn.ReLU(),
            #nn.LayerNorm(512),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.LayerNorm(256),
            nn.Linear(256, 12),
            #nn.Softmax(dim = -1)
        )
    def forward(self, x):
        result = self.linear_chain(x)
        return result

# model = NeuralNetwork().to(device)
# print(model)

Using cpu device.


In [5]:
# loss_fn = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# batch_size = 64

def train(x_train, y_train, model, loss_fn, optimizer):
    size = x_train.shape[0]
    for t in range(size // batch_size):
        X = x_train[t*batch_size:min((t+1)*batch_size, size)]
        y = y_train[t*batch_size:min((t+1)*batch_size, size)]
        
        pred = model(X).type(torch.float)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if t % 200 == 0:
            current = min((t+1)*batch_size, size)
            loss= loss.item()
            print(f"loss: {loss:>7f}, [ current: {current}/{size} ]")

In [6]:
def get_dcg(index):
    return 1 / np.log2(index + 2)

In [7]:
def get_ndcg(y_pred, y_true):
    ngdcg = 0.0
    
    for i in range(len(y_true)):
        index = np.where(y_pred[i] == y_true[i])[0]
        if len(index) == 0:
            continue
        ngdcg += get_dcg(index[0])
    return ngdcg / len(y_true)

# Xử lý tuổi với trung bình tuổi

In [8]:
transform_x_train = torch.tensor(data_train_avg.drop(["country_destination"], axis = 1).values, dtype=torch.float)
transform_y_train = torch.tensor(data_train_avg["country_destination"].values, dtype=torch.long)

transform_x_valid = torch.tensor(data_valid_avg.drop(["country_destination"], axis = 1).values, dtype = torch.float)
transform_y_valid = torch.tensor(data_valid_avg["country_destination"].values, dtype = torch.long)

transform_x_test = torch.tensor(data_test_avg.drop(["id"], axis = 1).values, dtype = torch.float)

In [9]:
epochs = 100
history_accuracy = [0]
t = 0

model = NeuralNetwork().to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
batch_size = 64

while True:
    print(f"Epochs {t+1}\n.....................................")
    t+=1
    train(transform_x_train, transform_y_train, model, loss_fn, optimizer)
    pred = model(transform_x_valid)
    pred = torch.argsort(pred, dim = 1, descending = True)[:, :5]
    ndcg = get_ndcg(pred, transform_y_valid)
    history_accuracy.append(ndcg)
    print(f"NDCG_score on validation set: {ndcg.item():>7f}")
    
    if(history_accuracy[-1] - history_accuracy[-2] < 0.00001 or t > epochs):
        break
print("Done!")

Epochs 1
.....................................
loss: 3.049409, [ current: 64/170760 ]
loss: 1.215357, [ current: 12864/170760 ]
loss: 1.058941, [ current: 25664/170760 ]
loss: 1.104049, [ current: 38464/170760 ]
loss: 1.125695, [ current: 51264/170760 ]
loss: 1.010108, [ current: 64064/170760 ]
loss: 1.265569, [ current: 76864/170760 ]
loss: 0.954996, [ current: 89664/170760 ]
loss: 1.085987, [ current: 102464/170760 ]
loss: 0.896758, [ current: 115264/170760 ]
loss: 1.313944, [ current: 128064/170760 ]
loss: 1.198613, [ current: 140864/170760 ]
loss: 1.191413, [ current: 153664/170760 ]
loss: 1.097276, [ current: 166464/170760 ]
NDCG_score on validation set: 0.809675
Epochs 2
.....................................
loss: 0.995764, [ current: 64/170760 ]
loss: 1.179292, [ current: 12864/170760 ]
loss: 1.052679, [ current: 25664/170760 ]
loss: 1.084426, [ current: 38464/170760 ]
loss: 1.117171, [ current: 51264/170760 ]
loss: 1.000906, [ current: 64064/170760 ]
loss: 1.275655, [ current: 

loss: 1.183119, [ current: 12864/170760 ]
loss: 1.027892, [ current: 25664/170760 ]
loss: 1.034380, [ current: 38464/170760 ]
loss: 1.088295, [ current: 51264/170760 ]
loss: 0.980875, [ current: 64064/170760 ]
loss: 1.319783, [ current: 76864/170760 ]
loss: 0.882500, [ current: 89664/170760 ]
loss: 1.001179, [ current: 102464/170760 ]
loss: 0.812552, [ current: 115264/170760 ]
loss: 1.277433, [ current: 128064/170760 ]
loss: 1.151623, [ current: 140864/170760 ]
loss: 1.170333, [ current: 153664/170760 ]
loss: 1.040897, [ current: 166464/170760 ]
NDCG_score on validation set: 0.815214
Epochs 14
.....................................
loss: 1.011792, [ current: 64/170760 ]
loss: 1.183126, [ current: 12864/170760 ]
loss: 1.025768, [ current: 25664/170760 ]
loss: 1.033113, [ current: 38464/170760 ]
loss: 1.084616, [ current: 51264/170760 ]
loss: 0.981058, [ current: 64064/170760 ]
loss: 1.321798, [ current: 76864/170760 ]
loss: 0.879199, [ current: 89664/170760 ]
loss: 0.997975, [ current: 1

In [10]:
for ind, item in enumerate(history_accuracy):
    print(f"Epoch {ind}:\n \t NDCG score: {item}")

Epoch 0:
 	 NDCG score: 0
Epoch 1:
 	 NDCG score: 0.8096747125780948
Epoch 2:
 	 NDCG score: 0.8107751330188879
Epoch 3:
 	 NDCG score: 0.8107901768070858
Epoch 4:
 	 NDCG score: 0.811519436002915
Epoch 5:
 	 NDCG score: 0.8121658123798162
Epoch 6:
 	 NDCG score: 0.8127901230451857
Epoch 7:
 	 NDCG score: 0.8132448200362937
Epoch 8:
 	 NDCG score: 0.8135229080219908
Epoch 9:
 	 NDCG score: 0.8137964140543745
Epoch 10:
 	 NDCG score: 0.8144345188843124
Epoch 11:
 	 NDCG score: 0.8146708108815357
Epoch 12:
 	 NDCG score: 0.8149906815596152
Epoch 13:
 	 NDCG score: 0.8152139828360705
Epoch 14:
 	 NDCG score: 0.8156841534526785
Epoch 15:
 	 NDCG score: 0.8160979008338254
Epoch 16:
 	 NDCG score: 0.8162969328147373
Epoch 17:
 	 NDCG score: 0.8167909858242998
Epoch 18:
 	 NDCG score: 0.8173683514900905
Epoch 19:
 	 NDCG score: 0.8174482405772955
Epoch 20:
 	 NDCG score: 0.8179140226918855
Epoch 21:
 	 NDCG score: 0.8181382163280022
Epoch 22:
 	 NDCG score: 0.8180828196262575


In [11]:
id_user = np.array(data_test_avg["id"])

pred_avg = model(transform_x_test)
pred_avg = np.array(torch.argsort(pred_avg, dim = 1, descending = True)[:, :5])

ids = []
country = []

dict_ = {'AU': 3,
 'CA': 9,
 'DE': 2,
 'ES': 8,
 'FR': 6,
 'GB': 7,
 'IT': 1,
 'NDF': 11,
 'NL': 10,
 'PT': 5,
 'US': 4,
 'other': 0}

inverse_dict = {v: k for k,v in dict_.items()}

for i in range(len(id_user)):
    ids += [id_user[i]]*5
    for j in range(5):
        country.append(inverse_dict[pred_avg[i, j]])

In [12]:
result_avg = {"id": ids, "country": country}

submission_avg = pd.DataFrame(result_avg)

submission_avg.to_csv("submission_avg.csv", index=False)

# Xử lý tuổi với kỹ thuật KNN

In [13]:
transform_x_train_knn = torch.tensor(data_train_knn.drop(["country_destination"], axis = 1).values, dtype=torch.float)
transform_y_train_knn = torch.tensor(data_train_knn["country_destination"].values, dtype=torch.long)

transform_x_valid_knn = torch.tensor(data_valid_knn.drop(["country_destination"], axis = 1).values, dtype = torch.float)
transform_y_valid_knn = torch.tensor(data_valid_knn["country_destination"].values, dtype = torch.long)

transform_x_test_knn = torch.tensor(data_test_knn.drop(["id"], axis = 1).values, dtype = torch.float)

In [14]:
epochs = 100
hist_knn = [0.0]
t = 0

model_knn = NeuralNetwork().to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_knn.parameters(), lr = 1e-3)
batch_size = 64

while True:
    print(f"Epochs {t+1}\n.....................................")
    t+=1
    train(transform_x_train_knn, transform_y_train_knn, model_knn, loss_fn, optimizer)
    pred = model_knn(transform_x_valid_knn)
    pred = torch.argsort(pred, dim = 1, descending = True)[:, :5]
    ndcg = get_ndcg(pred, transform_y_valid_knn)
    hist_knn.append(ndcg)
    print(f"NDCG_score on validation set: {ndcg.item():>7f}")
    
    if(hist_knn[-1] - hist_knn[-2] < 0.00001 or t > epochs):
        break
print("Done!")

Epochs 1
.....................................
loss: 2.872881, [ current: 64/170760 ]
loss: 1.463033, [ current: 12864/170760 ]
loss: 1.230677, [ current: 25664/170760 ]
loss: 0.966841, [ current: 38464/170760 ]
loss: 1.036481, [ current: 51264/170760 ]
loss: 0.925276, [ current: 64064/170760 ]
loss: 1.029187, [ current: 76864/170760 ]
loss: 0.898121, [ current: 89664/170760 ]
loss: 1.208408, [ current: 102464/170760 ]
loss: 0.958022, [ current: 115264/170760 ]
loss: 1.293929, [ current: 128064/170760 ]
loss: 1.089796, [ current: 140864/170760 ]
loss: 1.025426, [ current: 153664/170760 ]
loss: 1.143654, [ current: 166464/170760 ]
NDCG_score on validation set: 0.814909
Epochs 2
.....................................
loss: 1.118216, [ current: 64/170760 ]
loss: 1.385488, [ current: 12864/170760 ]
loss: 1.205580, [ current: 25664/170760 ]
loss: 0.955065, [ current: 38464/170760 ]
loss: 1.027006, [ current: 51264/170760 ]
loss: 0.901653, [ current: 64064/170760 ]
loss: 1.017020, [ current: 

In [15]:
id_user = np.array(data_test_knn["id"])

pred_knn = model_knn(transform_x_test_knn)
pred_knn = np.array(torch.argsort(pred_knn, dim = 1, descending = True)[:, :5])

ids = []
country = []

dict_ = {'AU': 3,
 'CA': 9,
 'DE': 2,
 'ES': 8,
 'FR': 6,
 'GB': 7,
 'IT': 1,
 'NDF': 11,
 'NL': 10,
 'PT': 5,
 'US': 4,
 'other': 0}

inverse_dict = {v: k for k,v in dict_.items()}

for i in range(len(id_user)):
    ids += [id_user[i]]*5
    for j in range(5):
        country.append(inverse_dict[pred_knn[i, j]])

In [21]:
result_knn = {"id": ids, "country": country}

submission_knn = pd.DataFrame(result_knn)

submission_knn.to_csv("submission_knn.csv", index=False)

In [19]:
print ("ndcg_score on validation set: {}, kaggle_private_score: 0.85994".format(history_accuracy[-1]))

ndcg_score on validation set: 0.8180828196262575, kaggle_private_score: 0.85994


In [22]:
print ("ndcg_score on validation set: {}, kaggle_private_score: 0.86628".format(hist_knn[-1]))

ndcg_score on validation set: 0.8222729266441533, kaggle_private_score: 0.86628
